# Создание нейронной сети без фреймворков

**Цели**:
* Создать модель классификации, не используя PyTorch или TensorFlow.
* Получить accuracy модели не ниже 0.9.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, f1_score

In [2]:
pd.options.display.float_format = '{:0.3f}'.format

## Загрузка данных

In [3]:
url = 'https://code.s3.yandex.net/datasets/'

In [4]:
df = pd.read_csv(url + 'insurance.csv')
display(df.head())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.000,49600.000,1,0
1,0,46.000,38000.000,1,1
2,0,29.000,21000.000,0,0
3,0,21.000,41700.000,2,0
4,1,28.000,26100.000,0,0


# Предобработка данных

Проверим наличие пропусков в данных.

In [5]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Сформируем признаки и выберем в качестве целевой переменной – "Страховые выплаты".

Сгруппируем данные чтобы свести задачу к бинарной.

In [6]:
df['Страховые выплаты'].value_counts()

0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

In [7]:
df['insurance'] = df['Страховые выплаты'] > 0
df.drop(columns='Страховые выплаты', inplace=True)

In [8]:
df['insurance'].value_counts()

False    4436
True      564
Name: insurance, dtype: int64

# Построение модели

Разделим выборку на тренировочную и тестовую.

In [9]:
features = df.drop('insurance', axis=1)
target = df['insurance']

X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.25, random_state=38)

Создадим класс нейронной сети, реализовав стандартные методы fit и predict.

In [10]:
class CustomNN:
    def __init__(self):
        # Зададим альфа-коэффициент чтобы избежать избыточной коррекции весов.
        self.alpha = 0.01
        # Ограничим число итераций.
        self.iterations = 120
        # Установим величину скрытого слоя.
        self.hidden_size = 32
        return None
    
    # Чтобы у модели появилась нелинейность используем ReLU.
    def relu(self, k):
        return k * (k > 0)
    # Производная от ReLU.
    def relu_derivative(self, k):
        return k > 0
    # Функция активации для выходного слоя.
    def sigmoid(self, k):
        return 1 / (1 + np.exp(-k))
    
    def fit(self, X, y):        
        X = np.array(X)
        # Преобразуем "y" в вектор с 1 столбцом.
        y = np.array(y).reshape(-1, 1)
        
        np.random.seed(38)
        
        # Инициируем веса для слоев.
        epsilon_init = 1
        w_0_1 = epsilon_init * np.random.random((X.shape[1], self.hidden_size)) - epsilon_init*0.5
        w_1_2 = epsilon_init * np.random.random((self.hidden_size, y.shape[1])) - epsilon_init*0.5
        
        for j in range(self.iterations):
            for i in range(X.shape[0]):
                # Первый слой - входные признаки.
                layer_0 = X[[i]]
                
                # Скалярное произведение первого слоя и весов.
                # Общая схема матричных операций 1x4 @ 4x32 @ 32x1
                layer_1 = self.relu(layer_0 @ w_0_1)
                # Применем регуляризацию чтобы модель не переобучилась.
                # После умножения слоя 1 на маску дополнительно умножаем на 2 для усиления.
                dropout = np.random.randint(2, size=layer_1.shape)  
                layer_1 *= dropout * 2
                layer_2 = self.sigmoid(layer_1 @ w_1_2)

                # Посчитаем веса используя градиентный спуск.
                layer_2_delta = y[[i]] - np.round(layer_2)
                layer_1_delta = (layer_2_delta @ w_1_2.T) * self.relu_derivative(layer_1)
                layer_1_delta *= dropout
                
                # Обратное распространение.
                w_1_2 += self.alpha * layer_1.T @ layer_2_delta
                w_0_1 += self.alpha * layer_0.T @ layer_1_delta
            
            if j%5 == 0:
                # Раз в 5 итераций будем обновлять F1 метрику для контроля переобучения.
                y_hat = self.sigmoid(self.relu(X @ w_0_1) @ w_1_2) > 0.5
                print(f'\rF1 Score на {j} итерации: {f1_score(y, y_hat):.3f}'.rjust(16, ' '), end='')
        
        self.w_0_1 = w_0_1
        self.w_1_2 = w_1_2
        return self

    def predict(self, X):
        X = np.array(X)
        
        # Возвращаем предсказания.
        return self.sigmoid(self.relu(X @ self.w_0_1) @ self.w_1_2) > 0.5

Составим pipeline и выведем отчет с основными метриками.

In [11]:
pipe_nn = make_pipeline(StandardScaler(), CustomNN())
y_hat = pipe_nn.fit(X_train, y_train).predict(X_test)
print('\n',classification_report(y_test, y_hat))

F1 Score на 115 итерации: 0.923
               precision    recall  f1-score   support

       False       1.00      1.00      1.00      1106
        True       0.99      1.00      0.99       144

    accuracy                           1.00      1250
   macro avg       0.99      1.00      1.00      1250
weighted avg       1.00      1.00      1.00      1250



# Выводы

* Получена нейронная сеть, позволяющая эффективно решать задачу классификации.
* Достигнута требуемая метрика качества, удалось избежать переобучения модели.